In [427]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import warnings
warnings.filterwarnings('ignore')


In [428]:
# Twitter Data
twitter_folder = "/Users/anshulrana/Desktop/vsc/python/Predicting_Success_Of_Startups/twitter_Data/tweets_data"

# Collating File Paths
twitter_file_paths = []
for i in os.listdir(twitter_folder):
    file_path = os.path.join(twitter_folder, i)
    if ".csv" in file_path:
        twitter_file_paths.append(file_path)



In [429]:
#Funding Data
funding_folder = "/Users/anshulrana/Desktop/vsc/python/Predicting_Success_Of_Startups/funding_Data"
#Funding File Path 
funding_file = os.path.join(funding_folder, "startup_funding_cleaned.csv")

funding_df = pd.read_csv(funding_file)
funding_df['Company'] = funding_df['Company'].str.lower()



In [430]:
import pandas as pd
import ast
import os
import re
from nltk.corpus import stopwords

# Function to combine datasets
def combine_datasets(twitter_data_path, company_name, combined_data):
    twitter_data = pd.read_csv(twitter_data_path)

    # Twitter Data Analysis
    if 'hashtags' in twitter_data.columns:
        twitter_data['hashtags'] = twitter_data['hashtags'].apply(ast.literal_eval)
        avg_hashtags = twitter_data['hashtags'].apply(len).mean()
    else:
        avg_hashtags = 0

    def count_mentions(tweet):
        mentions = re.findall(r'@(\w+)', tweet)
        return len(mentions)

    def count_links(tweet):
        links = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', tweet)
        return len(links)

    replies = twitter_data['replies'].mean()
    avg_mentions = twitter_data['tweet'].apply(count_mentions).mean()
    avg_tweet_length = twitter_data['tweet'].apply(len).mean()
    avg_likes = twitter_data['likes'].mean() if 'likes' in twitter_data.columns else 0
    avg_retweets = twitter_data['retweets'].mean() if 'retweets' in twitter_data.columns else 0
    avg_senti_score = twitter_data['senti_score'].mean() if 'senti_score' in twitter_data.columns else 0
    avg_links_mentioned = twitter_data['tweet'].apply(count_links).mean()

    # Twitter metrics DataFrame
    twitter_metrics = pd.DataFrame({
        "Company": company_name.lower(),
        "avg_likes": [avg_likes],
        "avg_retweets": [avg_retweets],
        "avg_hashtags": [avg_hashtags],
        "avg_senti": [avg_senti_score],
        "avg_mentions": [avg_mentions],
        "avg_tweet_length": [avg_tweet_length],
        "avg_links_mentioned": [avg_links_mentioned],
        "avg_replies": [replies]
    })


    # 
    combined_data = pd.concat([combined_data, twitter_metrics], ignore_index=True)
    return combined_data



In [431]:
# Initialize an empty DataFrame for combined data
combined_all_data = pd.DataFrame()

# Paths to the data files
funding_data_path = "/Users/anshulrana/Desktop/vsc/python/Predicting_Success_Of_Startups/funding_Data/startup_funding_cleaned.csv" # Update this with the correct path
twitter_folder = "/Users/anshulrana/Desktop/vsc/python/Predicting_Success_Of_Startups/twitter_Data/tweets_data"   # Update this with the correct path

for i in os.listdir(twitter_folder):
    file_path = os.path.join(twitter_folder, i)
    if ".csv" in file_path:
        company_name = i.split(".")[0]
        combined_all_data = combine_datasets(file_path, company_name, combined_all_data)
        combined_all_data['Company'] = combined_all_data['Company'].str.lower()

# Forward fill the 'Company' column to associate the values correctly
combined_all_data['Company'] = combined_all_data['Company'].ffill()
# Now we will drop duplicate rows for each company, keeping the first occurrence
cleaned_data = combined_all_data.drop_duplicates(subset=['Company'], keep='first')

In [432]:
cleaned_data.to_csv("combined_data.csv", index=False)

In [433]:
combined_df = pd.merge(cleaned_data, funding_df, on='Company', how='left')
combined_df.to_csv("final_data.csv", index=False)

In [434]:
# For the repeating companies, retain the one with the smaller value of year
combined_df['Year'] = combined_df['Year'].astype(int)
combined_df = combined_df.sort_values(by=['Year'], ascending=True)
combined_df = combined_df.drop_duplicates(subset=['Company'], keep='first')
combined_df.to_csv("naya_data.csv", index=False)


In [435]:
#Including market analysis data
market_folder = "/Users/anshulrana/Desktop/vsc/python/Predicting_Success_Of_Startups/market_analysis"
gdp = pd.read_csv(os.path.join(market_folder, "gdp_data.csv"))

#Add another column of GDP that adds the GDP growth rate for the value of the Year
combined_df['Growth'] = combined_df['Year'].apply(lambda x: gdp[gdp['Year'] == x]['Growth'].values[0])

combined_df.to_csv('bleh.csv',index=False)


In [437]:
#Include fdi data
fdi = pd.read_csv(os.path.join(market_folder, "india_fdi_data.csv"))
combined_df['FDI'] = combined_df['Year'].apply(lambda x: fdi[fdi['Year'] == x]['FDI Net Inflows (% of GDP)'].values[0])

In [438]:
combined_df.to_csv('bleh.csv',index=False)